In [22]:
# set the matplotlib backend so figures can be saved in the background
#TODO: cleanup, lint
import matplotlib
matplotlib.use("Agg")
 
import pathlib
from pathlib import Path
import matplotlib.pyplot as plot
import librosa
import keras
from keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
    
# import the necessary packages
from musicrec.vgg import VGGNet
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras import regularizers
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
from datetime import datetime
import pickle
import cv2
import os
import sys
import tensorflow as tf
import pickle
import json
import re

from keras.models import Sequential

import warnings

#env parameters
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [23]:
# Paramters
data_folder = Path("../../../audio/testfiles/GTZAN/genres/")
output_root = './../../../models/'
output_folder = Path('./output/cvnn.model')
output_model = output_root + '/cnn_dong_model_weights.h5'
output_architecture = output_root + '/cnn_dong_model_architecture.json'
output_whole = output_root + 'cnn_dong_model_whole.h5'
output_best_model = output_root + 'best_model.h5'
output_label = output_root + 'label.pkl'
output_test_paths = output_root + 'test_paths.pkl'
spectogram_folder = Path("./img_data/")
# Duration of songsnippet in seconds
duration = 2.97
start_offset = 3
# Matplotlib colormap for spectogram
spectogram_cmap = 'binary' 
# Predefined list of genres
pred_genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split() 
epochs = 400
num_segments = 4

es_patience = 200
return_train_and_test = 1

# parameters
sr = 22050 # if sampling rate is different, resample it to this

# parameters for calculating spectrogram in mel scale
fmax = 1500 # maximum frequency considered
fft_window_points = 512
fft_window_dur = fft_window_points * 1.0 / sr # 23ms windows
hop_size = int(fft_window_points/ 2) # 50% overlap between consecutive frames
n_mels = 64

# segment duration
num_fft_windows = 256 # num fft windows per music segment
segment_in_points = num_fft_windows * 255 # number of data points that ensure the spectrogram has size: 64 * 256
segment_dur = segment_in_points * 1.0 / sr

num_genres=10
input_shape=(64, 256, 1)

randomseed = 11
#randomseed = datetime.now()
# Seed for RNG
random.seed(randomseed)


In [24]:
#Get directories of all songs
songs = []
genres = []

for g in data_folder.iterdir():
    genres.append(g.name)
    for i in g.iterdir():
        songs.append(i)

In [25]:
mel_specs = []  
labels = []

#loading with different numbers of segements for data-augmentation
#!Initialize "spectograms" before running
def load_specs(data = songs, num_segments = 1):
    for song in data:
        offset = start_offset
        for i in range(num_segments):
            y, sr = librosa.load(song, mono=True, offset=offset, duration=duration)
            m_sp = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=fft_window_points,
                                                  hop_length=hop_size, n_mels=n_mels,
                                                  fmax=fmax)
        #if m_sp.shape != (128, 128): continue
            mel_specs.append([m_sp])
            label = song.parts[-2]
            labels.append(label)
            spectograms.append( (m_sp, label, song, offset, duration) )
            offset = offset + duration/2
        input_shape = m_sp.shape + (1,)

In [26]:
#shuffle songs to keep segements together
random.shuffle(songs)

[PosixPath('../../../audio/testfiles/GTZAN/genres/blues/blues.00011.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/blues/blues.00064.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/country/country.00009.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/disco/disco.00091.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/classical/classical.00099.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/metal/metal.00002.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/blues/blues.00024.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/country/country.00043.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/rock/rock.00065.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/pop/pop.00002.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/blues/blues.00099.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/disco/disco.00081.wav'), PosixPath('../../../audio/testfiles/GTZAN/genres/hiphop/hiphop.00088.wav'), PosixPath('../../../audio

In [27]:
spectograms = []
load_specs(data = songs, num_segments = num_segments)
len(spectograms)

4000

In [28]:
print("Total number of samples: ", len(mel_specs))

#set dynamic input shape
input_shape = np.shape(mel_specs[2]) + (1,)
input_shape = (input_shape[1], input_shape[2], input_shape[3])
print("Input shape: " + str(input_shape))
np.shape(mel_specs)

Total number of samples:  4000
Input shape: (64, 256, 1)


(4000, 1, 64, 256)

In [29]:
batch_size = input_shape[1] # use second dim as batch size, as it varies with duration

In [30]:
imagesize_x = np.shape(mel_specs)[1]
imagesize_y = np.shape(mel_specs)[2]
print("Spectogram dimensions: ", imagesize_x, "x", imagesize_y)

Spectogram dimensions:  1 x 64


In [31]:
testsplit = len(spectograms)*0.7    #70% train-test-split
train = spectograms[:int(testsplit)]
test = spectograms[int(testsplit):]
np.shape(test)

(1200, 5)

In [32]:
#TODO: change


X_train, y_train, p_train, offset_train, duration_train = zip(*train)
X_test, y_test, p_test, offset_test, duration_test = zip(*test)

# Reshape for CNN input
X_train = np.array([x.reshape(input_shape) for x in X_train])
X_test = np.array([x.reshape(input_shape) for x in X_test])

# One-Hot encoding for classes
lb = LabelBinarizer()
y_train_bin = lb.fit_transform(y_train)
y_test_bin = lb.transform(y_test)

In [33]:
with open(output_label, 'wb') as f:
    pickle.dump(lb, f)
    
if return_train_and_test == 1:
    r_paths = p_test + (p_train)
    r_offsets = offset_test + (offset_train)
    r_durations = duration_test + (duration_train)
else:
    r_paths = p_test
    r_offsets = offset_test
    r_durations = duration_test
    
r_values = [r_paths, r_offsets, r_durations]

with open(output_test_paths, 'wb') as f:
    pickle.dump(r_values, f)

3

In [34]:
y_train = y_train_bin
y_test = y_test_bin

In [36]:
#old model for urban
def cnn_urban_model_build():
    model = Sequential()

    model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape))
    model.add(MaxPooling2D((4, 2), strides=(4, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(48, (5, 5), padding="valid"))
    model.add(MaxPooling2D((4, 2), strides=(4, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(48, (5, 5), padding="valid"))
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dropout(rate=0.5))

    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(rate=0.5))

    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(
        optimizer="Adam",
        loss="categorical_crossentropy",
        metrics=['accuracy'])
    print(model.summary)
    return model

#model for GTZAN
def cnn_dong_model_build():

    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3, 3),
                     activation='relu', kernel_regularizer=regularizers.l2(0.01),
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 4)))

    model.add(Conv2D(64, (3, 5), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(MaxPooling2D(pool_size=(2, 4)))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.02)))
    model.add(Dropout(0.3))
    model.add(Dense(num_genres, activation='softmax'))
    #model.compile(loss=keras.losses.categorical_crossentropy,
     #             optimizer=keras.optimizers.Adadelta(decay=1e-6),
      #            metrics=['accuracy'])
    model.compile(
        optimizer="Adam",
        loss="categorical_crossentropy",
        metrics=['accuracy'])
    print(model.summary)
    return model

In [37]:
model = cnn_dong_model_build()

<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x7f17882326a0>>


In [38]:
callbacks = [EarlyStopping(monitor='val_loss', patience=es_patience),
             ModelCheckpoint(filepath=output_best_model, monitor='val_loss', save_best_only=True)]

H = model.fit(
	x=X_train, 
	y=y_train,
    epochs=epochs,
    callbacks=callbacks,
    batch_size=batch_size,
    validation_data= (X_test, y_test))

score = model.evaluate(x=X_test,y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 2800 samples, validate on 1200 samples
Epoch 1/400
2800/2800 [==============================] - 85s 30ms/step - loss: 4.3128 - acc: 0.1168 - val_loss: 3.8321 - val_acc: 0.1308
Epoch 2/400
2800/2800 [==============================] - 84s 30ms/step - loss: 3.6316 - acc: 0.1539 - val_loss: 3.4218 - val_acc: 0.1700
Epoch 3/400
2800/2800 [==============================] - 84s 30ms/step - loss: 3.2767 - acc: 0.1811 - val_loss: 3.1674 - val_acc: 0.1575
Epoch 4/400
2800/2800 [==============================] - 84s 30ms/step - loss: 3.0162 - acc: 0.2064 - val_loss: 2.9764 - val_acc: 0.2075
Epoch 5/400
2800/2800 [==============================] - 84s 30ms/step - loss: 2.8364 - acc: 0.2464 - val_loss: 2.8220 - val_acc: 0.2300
Epoch 6/400
2800/2800 [==============================] - 84s 30ms/step - loss: 2.6823 - acc: 0.2896 - val_loss: 2.6506 - val_acc: 0.2767
Epoch 7/400
2800/2800 [==============================] - 84s 30ms/step - loss: 2.4956 - acc: 0.3146 - val_loss: 2.5235 - val_acc: 

2800/2800 [==============================] - 87s 31ms/step - loss: 1.4362 - acc: 0.7200 - val_loss: 2.4948 - val_acc: 0.4775
Epoch 61/400
2800/2800 [==============================] - 84s 30ms/step - loss: 1.4460 - acc: 0.7136 - val_loss: 2.6117 - val_acc: 0.4642
Epoch 62/400
2800/2800 [==============================] - 84s 30ms/step - loss: 1.3952 - acc: 0.7307 - val_loss: 2.5645 - val_acc: 0.4808
Epoch 63/400
2800/2800 [==============================] - 87s 31ms/step - loss: 1.3995 - acc: 0.7271 - val_loss: 2.5781 - val_acc: 0.4717
Epoch 64/400
2800/2800 [==============================] - 84s 30ms/step - loss: 1.4119 - acc: 0.7243 - val_loss: 2.4935 - val_acc: 0.4725
Epoch 65/400
2800/2800 [==============================] - 84s 30ms/step - loss: 1.3927 - acc: 0.7304 - val_loss: 2.6037 - val_acc: 0.4583
Epoch 66/400
2800/2800 [==============================] - 84s 30ms/step - loss: 1.3845 - acc: 0.7321 - val_loss: 2.4384 - val_acc: 0.4858
Epoch 67/400
2800/2800 [=======================

2800/2800 [==============================] - 84s 30ms/step - loss: 1.2385 - acc: 0.8046 - val_loss: 2.6946 - val_acc: 0.4917
Epoch 120/400
2800/2800 [==============================] - 88s 31ms/step - loss: 1.2566 - acc: 0.7964 - val_loss: 2.7061 - val_acc: 0.4933
Epoch 121/400
2800/2800 [==============================] - 84s 30ms/step - loss: 1.2594 - acc: 0.7886 - val_loss: 2.6730 - val_acc: 0.4833
Epoch 122/400
2800/2800 [==============================] - 83s 30ms/step - loss: 1.2817 - acc: 0.7896 - val_loss: 2.7074 - val_acc: 0.4792
Epoch 123/400
2800/2800 [==============================] - 84s 30ms/step - loss: 1.2628 - acc: 0.7989 - val_loss: 2.7578 - val_acc: 0.4808
Epoch 124/400
2800/2800 [==============================] - 87s 31ms/step - loss: 1.2446 - acc: 0.8075 - val_loss: 2.6583 - val_acc: 0.5000
Epoch 125/400
2800/2800 [==============================] - 84s 30ms/step - loss: 1.2514 - acc: 0.7964 - val_loss: 2.6970 - val_acc: 0.4933
Epoch 126/400
2800/2800 [================

2800/2800 [==============================] - 85s 30ms/step - loss: 1.0863 - acc: 0.8482 - val_loss: 2.6060 - val_acc: 0.5075
Epoch 179/400
2800/2800 [==============================] - 84s 30ms/step - loss: 1.0956 - acc: 0.8454 - val_loss: 2.7524 - val_acc: 0.5008
Epoch 180/400
2800/2800 [==============================] - 84s 30ms/step - loss: 1.0878 - acc: 0.8529 - val_loss: 2.6083 - val_acc: 0.4858
Epoch 181/400
2800/2800 [==============================] - 84s 30ms/step - loss: 1.1018 - acc: 0.8468 - val_loss: 2.7793 - val_acc: 0.4817
Epoch 182/400
2800/2800 [==============================] - 84s 30ms/step - loss: 1.1308 - acc: 0.8350 - val_loss: 2.7613 - val_acc: 0.4892
Epoch 183/400
2800/2800 [==============================] - 84s 30ms/step - loss: 1.1291 - acc: 0.8336 - val_loss: 2.6180 - val_acc: 0.5000
Epoch 184/400
2800/2800 [==============================] - 84s 30ms/step - loss: 1.1529 - acc: 0.8279 - val_loss: 2.6527 - val_acc: 0.4858
Epoch 185/400
2800/2800 [================

In [39]:
def evaluate_song(song):
    start = 0
    r = 0
    for i in range(int(30/(duration/2))-1):
        y, sr = librosa.load(song, mono=True, offset=start, duration=duration)
        m_sp = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=fft_window_points,
                                              hop_length=hop_size, n_mels=n_mels,
                                              fmax=fmax)
        m_sp = np.expand_dims(m_sp, 0)
        m_sp = np.expand_dims(m_sp, 3)
        if r == 0:
            prediction = model.predict(m_sp)
            r = 1
        else:
            prediction = (prediction + model.predict(m_sp))/2
        start = start + duration/2
    return prediction
        
def evaluate_batch(batch):
    predictions = []
    for song in batch:
        predictions.append(evaluate_song(song))
    return predictions

def compare_batch(predictions):
    i = 0
    l = lb.inverse_transform(y_test)
    counter = 0
    for prediction in predictions:
        tr_value = str(l[i])
        pr_value = re.sub('[\[\]\']', '', str(lb.classes_[prediction.argmax(axis=1)]))
        print(tr_value + "=>" + pr_value)
        if tr_value == pr_value:
            counter += 1
        i += 1
    acc = counter/len(predictions)*100
    print("Acc: " + str(acc) + "%")

In [40]:
predictions = evaluate_batch(p_test)
print(predictions)

[array([[2.4264427e-03, 1.0306860e-04, 5.7557062e-04, 2.7383167e-02,
        9.9385634e-02, 2.3868913e-02, 2.0086097e-03, 3.2035485e-02,
        8.0976152e-01, 2.4515418e-03]], dtype=float32), array([[2.4264427e-03, 1.0306860e-04, 5.7557062e-04, 2.7383167e-02,
        9.9385634e-02, 2.3868913e-02, 2.0086097e-03, 3.2035485e-02,
        8.0976152e-01, 2.4515418e-03]], dtype=float32), array([[2.4264427e-03, 1.0306860e-04, 5.7557062e-04, 2.7383167e-02,
        9.9385634e-02, 2.3868913e-02, 2.0086097e-03, 3.2035485e-02,
        8.0976152e-01, 2.4515418e-03]], dtype=float32), array([[2.4264427e-03, 1.0306860e-04, 5.7557062e-04, 2.7383167e-02,
        9.9385634e-02, 2.3868913e-02, 2.0086097e-03, 3.2035485e-02,
        8.0976152e-01, 2.4515418e-03]], dtype=float32), array([[0.01887888, 0.00045424, 0.00069215, 0.2296715 , 0.30628923,
        0.00077643, 0.06798047, 0.08111869, 0.25526923, 0.03886915]],
      dtype=float32), array([[0.01887888, 0.00045424, 0.00069215, 0.2296715 , 0.30628923,
   

In [41]:
compare_batch(predictions)

reggae=>reggae
reggae=>reggae
reggae=>reggae
reggae=>reggae
reggae=>hiphop
reggae=>hiphop
reggae=>hiphop
reggae=>hiphop
hiphop=>pop
hiphop=>pop
hiphop=>pop
hiphop=>pop
pop=>pop
pop=>pop
pop=>pop
pop=>pop
rock=>reggae
rock=>reggae
rock=>reggae
rock=>reggae
classical=>classical
classical=>classical
classical=>classical
classical=>classical
hiphop=>hiphop
hiphop=>hiphop
hiphop=>hiphop
hiphop=>hiphop
hiphop=>metal
hiphop=>metal
hiphop=>metal
hiphop=>metal
jazz=>blues
jazz=>blues
jazz=>blues
jazz=>blues
rock=>rock
rock=>rock
rock=>rock
rock=>rock
jazz=>jazz
jazz=>jazz
jazz=>jazz
jazz=>jazz
hiphop=>hiphop
hiphop=>hiphop
hiphop=>hiphop
hiphop=>hiphop
rock=>metal
rock=>metal
rock=>metal
rock=>metal
reggae=>reggae
reggae=>reggae
reggae=>reggae
reggae=>reggae
reggae=>reggae
reggae=>reggae
reggae=>reggae
reggae=>reggae
disco=>disco
disco=>disco
disco=>disco
disco=>disco
rock=>rock
rock=>rock
rock=>rock
rock=>rock
reggae=>disco
reggae=>disco
reggae=>disco
reggae=>disco
classical=>classical
classic

rock=>rock
rock=>rock
metal=>rock
metal=>rock
metal=>rock
metal=>rock
classical=>classical
classical=>classical
classical=>classical
classical=>classical
jazz=>jazz
jazz=>jazz
jazz=>jazz
jazz=>jazz
country=>country
country=>country
country=>country
country=>country
pop=>blues
pop=>blues
pop=>blues
pop=>blues
jazz=>rock
jazz=>rock
jazz=>rock
jazz=>rock
rock=>country
rock=>country
rock=>country
rock=>country
pop=>pop
pop=>pop
pop=>pop
pop=>pop
rock=>blues
rock=>blues
rock=>blues
rock=>blues
classical=>classical
classical=>classical
classical=>classical
classical=>classical
reggae=>jazz
reggae=>jazz
reggae=>jazz
reggae=>jazz
pop=>hiphop
pop=>hiphop
pop=>hiphop
pop=>hiphop
country=>country
country=>country
country=>country
country=>country
classical=>classical
classical=>classical
classical=>classical
classical=>classical
country=>blues
country=>blues
country=>blues
country=>blues
metal=>metal
metal=>metal
metal=>metal
metal=>metal
pop=>hiphop
pop=>hiphop
pop=>hiphop
pop=>hiphop
jazz=>jazz

In [42]:
# evaluate the network
print("[INFO] evaluating network...")
#predictions = model.predict(X_test, batch_size=batch_size)



#print(classification_report(y_test_bin.argmax(axis=1),
#                            predictions.argmax(axis=1), 
#                            target_names=lb.classes_))
 
# plot the training loss and accuracy
N = np.arange(0, epochs)
plt.style.use("ggplot")
plt.figure()
"""
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (SmallVGGNet)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
"""
0
print(predictions)

[INFO] evaluating network...


AttributeError: 'list' object has no attribute 'argmax'

In [ ]:
cm = confusion_matrix(y_test_bin.argmax(axis=1), predictions.argmax(axis=1))

#cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm

In [ ]:
# Save the weights
#model.save_weights(output_model)

# Save the model architecture
#with open(output_architecture, 'w') as f:
#    f.write(model.to_json())
    
#model.save(output_whole)